In [ ]:
import cv2
from matplotlib import pyplot
import numpy
from scipy.fftpack import dct
from scipy.fftpack import idct
import struct
import zlib

# Zadanie 1

In [ ]:
#
# PPM file header
#
ppm_ascii_header = 'P3\n3 1\n255\n'  # TODO: implement
ppm_binary_header = 'P6\n3 1\n255\n'  # TODO: implement

#
# Image data
#
image = numpy.array([0, 255, 0, 0, 255, 0, 0, 255, 0], dtype=numpy.uint8)  # TODO: implement

#
# Save the PPM image as an ASCII file
#
with open('lab4-ascii.ppm', 'w') as fh:
    fh.write(ppm_ascii_header)
    image.tofile(fh, sep=' ')
    fh.write('\n')

#
# Save the PPM image as a binary file
#
with open('lab4-binary.ppm', 'wb') as fh:
    fh.write(bytearray(ppm_binary_header, 'ascii'))
    image.tofile(fh)

In [ ]:
#
# Display image
#
image_from_file = cv2.imread('lab4-ascii.ppm')
pyplot.imshow(cv2.cvtColor(image_from_file, cv2.COLOR_BGR2RGB))
pyplot.axis('off') 
pyplot.show()

In [ ]:
#
# Display image
#
image_from_file = cv2.imread('lab4-binary.ppm')
pyplot.imshow(cv2.cvtColor(image_from_file, cv2.COLOR_BGR2RGB))
pyplot.axis('off') 
pyplot.show()

# Zadanie 2

In [ ]:
#
# Image data
#

base_colors = [(0, 0, 0),        # black
               (0, 0, 255),      # blue
               (0, 255, 255),    # cyan
               (0, 255, 0),      # green
               (255, 255, 0),    # yellow 
               (255, 0, 0),      # red
               (255, 0, 255),    # purple
               (255, 255, 255)]  # white

# Number of interpolation steps between each color
steps = 10

# Generate gradient
gradient = []
for i in range(len(base_colors) - 1):
    start = numpy.array(base_colors[i])
    end = numpy.array(base_colors[i + 1])
    for t in range(steps):
        ratio = t / (steps - 1)
        color = (1 - ratio) * start + ratio * end
        gradient.append(color.astype(numpy.uint8))

# Convert to 2D image of shape (height, width, 3)
gradient_row = numpy.array(gradient, dtype=numpy.uint8)
width = gradient_row.shape[0]
height = 50

image = numpy.tile(gradient_row[numpy.newaxis, :, :], (height, 1, 1))  # turns (1, width, 3) into (height, width, 3)

# Flatten image to 1D for PPM ASCII output
image_flat = image.reshape(-1, 3)

#
# PPM file header
#
ppm_ascii_header = f'P3\n{width} {height}\n255\n'

#
# Save the PPM image as an ASCII file
#
with open('lab4-rainbow.ppm', 'w') as fh:
    fh.write(ppm_ascii_header)
    for pixel in image_flat:
        fh.write(f"{pixel[0]} {pixel[1]} {pixel[2]} ")
    fh.write('\n')



In [ ]:
#
# Display image
#
image_from_file = cv2.imread('lab4-rainbow.ppm')
pyplot.imshow(cv2.cvtColor(image_from_file, cv2.COLOR_BGR2RGB))
pyplot.axis('off') 
pyplot.show()

# Zadanie 3

In [ ]:
#
# Image data
#
#image = numpy.array([[255,   0,   0], [  0, 255,  0]],
#                    dtype=numpy.uint8)  # TODO: implement

base_colors = [(0, 0, 0),        # czarny
               (0, 0, 255),      # niebieski
               (0, 255, 255),    # blekitny
               (0, 255, 0),      # zielony
               (255, 255, 0),    # zolty 
               (255, 0, 0),      # czerwony
               (255, 0, 255),    # fioletowy
               (255, 255, 255)]  # bialy

# Number of interpolation steps between each color
steps = 10

# Generate gradient
gradient = []
for i in range(len(base_colors) - 1):
    start = numpy.array(base_colors[i])
    end = numpy.array(base_colors[i + 1])
    for t in range(steps):
        ratio = t / (steps - 1)
        color = (1 - ratio) * start + ratio * end
        gradient.append(color.astype(numpy.uint8))

gradient_row = numpy.array(gradient, dtype=numpy.uint8).reshape(1, -1, 3)
width = gradient_row.shape[1]
height = 50

# Repeat the same row multiple times
image = numpy.tile(gradient_row, (height, 1, 1))

scanlines = b''.join([b'\x00' + row.tobytes() for row in image])
compressed_data = zlib.compress(scanlines)

#
# Construct signature
#
png_file_signature = b'\x89PNG\r\n\x1a\n'  # TODO: implement

#
# Construct header
#
header_id = b'IHDR'  # TODO: implement
header_content = struct.pack('!2I5B', width, height, 8, 2, 0, 0, 0)  # ! - kolejnosc bajtow big endian
header_size = struct.pack('!I', len(header_content))  # TODO: implement
header_crc = struct.pack('!I', zlib.crc32(header_id + header_content))  # TODO: implement
png_file_header = header_size + header_id + header_content + header_crc

#
# Construct data
#
data_id = b'IDAT'  # TODO: implement
data_content = compressed_data  # TODO: implement
data_size = struct.pack('!I', len(data_content))  # TODO: implement
data_crc = struct.pack('!I', zlib.crc32(data_id + data_content))  # TODO: implement
png_file_data = data_size + data_id + data_content + data_crc

#
# Consruct end
#
end_id = b'IEND'
end_content = b''
end_size = struct.pack('!I', len(end_content))
end_crc = struct.pack('!I', zlib.crc32(end_id + end_content))
png_file_end = end_size + end_id + end_content + end_crc

#
# Save the PNG image as a binary file
#
with open('lab4.png', 'wb') as fh:
    fh.write(png_file_signature)
    fh.write(png_file_header)
    fh.write(png_file_data)
    fh.write(png_file_end)

In [ ]:
#
# Display image
#
image_from_file = cv2.imread('lab4.png')
pyplot.imshow(cv2.cvtColor(image_from_file, cv2.COLOR_BGR2RGB))
pyplot.axis('off') 
pyplot.show()

# Funkcje pomocnicze do zadań 4 i 5

In [ ]:
#
# 2d Discrete Cosinus Transform
#
def dct2(array):
    return dct(dct(array, axis=0, norm='ortho'), axis=1, norm='ortho')


def idct2(array):
    return idct(idct(array, axis=0, norm='ortho'), axis=1, norm='ortho')


#
# Calculate quantisation matrices
#
# Based on: https://www.hdm-stuttgart.de/~maucher/Python/MMCodecs/html/jpegUpToQuant.html
#           #step-3-and-4-discrete-cosinus-transform-and-quantisation
#
_QY = numpy.array([[16, 11, 10, 16, 24, 40, 51, 61],
                   [12, 12, 14, 19, 26, 48, 60, 55],
                   [14, 13, 16, 24, 40, 57, 69, 56],
                   [14, 17, 22, 29, 51, 87, 80, 62],
                   [18, 22, 37, 56, 68, 109, 103, 77],
                   [24, 35, 55, 64, 81, 104, 113, 92],
                   [49, 64, 78, 87, 103, 121, 120, 101],
                   [72, 92, 95, 98, 112, 100, 103, 99]])

_QC = numpy.array([[17, 18, 24, 47, 99, 99, 99, 99],
                   [18, 21, 26, 66, 99, 99, 99, 99],
                   [24, 26, 56, 99, 99, 99, 99, 99],
                   [47, 66, 99, 99, 99, 99, 99, 99],
                   [99, 99, 99, 99, 99, 99, 99, 99],
                   [99, 99, 99, 99, 99, 99, 99, 99],
                   [99, 99, 99, 99, 99, 99, 99, 99],
                   [99, 99, 99, 99, 99, 99, 99, 99]])


def _scale(QF):
    if QF < 50 and QF >= 1:
        scale = numpy.floor(5000 / QF)
    elif QF < 100:
        scale = 200 - 2 * QF
    else:
        raise ValueError('Quality Factor must be in the range [1..99]')

    scale = scale / 100.0
    return scale


def QY(QF=85):
    return _QY * _scale(QF)


def QC(QF=85):
    return _QC * _scale(QF)

# Zadania 4 i 5

In [ ]:
#
# 0. Image data
#
base_colors = [(0, 0, 0),        # czarny
               (0, 0, 255),      # niebieski
               (0, 255, 255),    # blekitny
               (0, 255, 0),      # zielony
               (255, 255, 0),    # zolty 
               (255, 0, 0),      # czerwony
               (255, 0, 255),    # fioletowy
               (255, 255, 255)]  # bialy

# Number of interpolation steps between each color
steps = 10
color_segments = len(base_colors) - 1

# Compute minimum width, round up to nearest multiple of 8
min_width = steps * color_segments
width = numpy.ceil(min_width / 8) * 8

# Generate gradient
gradient = []
for i in range(len(base_colors) - 1):
    start = numpy.array(base_colors[i])
    end = numpy.array(base_colors[i + 1])
    for t in range(steps):
        ratio = t / (steps - 1)
        color = (1 - ratio) * start + ratio * end
        gradient.append(color.astype(numpy.uint8))

# Pad horizontally if needed
gradient = numpy.array(gradient, dtype=numpy.uint8)
if len(gradient) < width:
    padding = numpy.tile(gradient[-1], (int(width - len(gradient)), 1))
    gradient = numpy.vstack((gradient, padding))

# Create row and tile vertically to height that's a multiple of 8
gradient_row = gradient.reshape(1, -1, 3)
height = 64
image_rgb = numpy.tile(gradient_row, (height, 1, 1))

cv2.imwrite("lab4-zad4-original.png", image_rgb)

#
# 1. Convert RGB to YCbCr
#
image_ycrcb = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2YCrCb)

#
# 2. Downsampling on Cb and Cr channels
#
Y, Cb, Cr = cv2.split(image_ycrcb)

Cb_downsampled_4_4_4 = Cb.copy()
Cr_downsampled_4_4_4 = Cr.copy()

Cb_downsampled_4_2_2 = Cb[:, ::2]
Cr_downsampled_4_2_2 = Cr[:, ::2]

Cb_downsampled_4_2_0 = Cb[::2, ::2]
Cr_downsampled_4_2_0 = Cr[::2, ::2]

#
# 3. Produce 8x8 blocks
#
def to_blocks(channel):
    h, w = channel.shape
    pad_h = (8 - h % 8) % 8
    pad_w = (8 - w % 8) % 8
    padded = numpy.pad(channel, ((0, pad_h), (0, pad_w)), 'constant')
    blocks = []
    for i in range(0, padded.shape[0], 8):
        for j in range(0, padded.shape[1], 8):
            blocks.append(padded[i:i+8, j:j+8])
    return numpy.array(blocks), padded.shape

Y_blocks, Y_shape = to_blocks(Y)

Cb_blocks_4, Cb_shape_4 = to_blocks(Cb_downsampled_4_4_4)
Cr_blocks_4, Cr_shape_4 = to_blocks(Cr_downsampled_4_4_4)

Cb_blocks_2, Cb_shape_2 = to_blocks(Cb_downsampled_4_2_2)
Cr_blocks_2, Cr_shape_2 = to_blocks(Cr_downsampled_4_2_2)

Cb_blocks_0, Cb_shape_0 = to_blocks(Cb_downsampled_4_2_0)
Cr_blocks_0, Cr_shape_0 = to_blocks(Cr_downsampled_4_2_0)


#
# 4. Calculate DCT on each block
#
Y_dct = numpy.array([dct2(block) for block in Y_blocks])

Cb_dct_4 = numpy.array([dct2(block) for block in Cb_blocks_4])
Cr_dct_4 = numpy.array([dct2(block) for block in Cr_blocks_4])

Cb_dct_2 = numpy.array([dct2(block) for block in Cb_blocks_2])
Cr_dct_2 = numpy.array([dct2(block) for block in Cr_blocks_2])

Cb_dct_0 = numpy.array([dct2(block) for block in Cb_blocks_0])
Cr_dct_0 = numpy.array([dct2(block) for block in Cr_blocks_0])

#
# 5. Divide each block by quantisation matrix
#

Y_quant = Y_dct / QY()

Cb_quant_4 = Cb_dct_4 / QC()
Cr_quant_4 = Cr_dct_4 / QC()

Cb_quant_2 = Cb_dct_2 / QC()
Cr_quant_2 = Cr_dct_2 / QC()

Cb_quant_0 = Cb_dct_0 / QC()
Cr_quant_0 = Cr_dct_0 / QC()

#
# 6. Round values in each block to integers
#

Y_quant_rounded = numpy.round(Y_quant).astype(numpy.int16)

Cb_quant_rounded_4 = numpy.round(Cb_quant_4).astype(numpy.int16)
Cr_quant_rounded_4 = numpy.round(Cr_quant_4).astype(numpy.int16)

Cb_quant_rounded_2 = numpy.round(Cb_quant_2).astype(numpy.int16)
Cr_quant_rounded_2 = numpy.round(Cr_quant_2).astype(numpy.int16)

Cb_quant_rounded_0 = numpy.round(Cb_quant_0).astype(numpy.int16)
Cr_quant_rounded_0 = numpy.round(Cr_quant_0).astype(numpy.int16)

#
# 7. Zig Zag
#
def zigzag(block):
    index_order = sorted(((x, y) for x in range(8) for y in range(8)),
                         key=lambda s: (s[0]+s[1], -s[1] if (s[0]+s[1]) % 2 else s[1]))
    return numpy.array([block[x, y] for x, y in index_order], dtype=numpy.int16)

Y_zigzag = numpy.array([zigzag(block) for block in Y_quant_rounded])

Cb_zigzag_4 = numpy.array([zigzag(block) for block in Cb_quant_rounded_4])
Cr_zigzag_4 = numpy.array([zigzag(block) for block in Cr_quant_rounded_4])

Cb_zigzag_2 = numpy.array([zigzag(block) for block in Cb_quant_rounded_2])
Cr_zigzag_2 = numpy.array([zigzag(block) for block in Cr_quant_rounded_2])

Cb_zigzag_0 = numpy.array([zigzag(block) for block in Cb_quant_rounded_0])
Cr_zigzag_0 = numpy.array([zigzag(block) for block in Cr_quant_rounded_0])


Y_zigzag_no_DCT = numpy.array([zigzag(block) for block in Y_blocks])
Cb_zigzag_no_DCT = numpy.array([zigzag(block) for block in Cb_blocks_2])
Cr_zigzag_no_DCT = numpy.array([zigzag(block) for block in Cr_blocks_2])

#
# 8. Flatten, concatenate, compress and calculate the size -- how many bytes?
#
combined_no_DCT = numpy.concatenate([Y_zigzag_no_DCT.flatten(), Cb_zigzag_no_DCT.flatten(), Cr_zigzag_no_DCT.flatten()])
compressed_no_DCT = zlib.compress(combined_no_DCT.tobytes())
compressed_size_no_DCT = len(compressed_no_DCT)

combined_4 = numpy.concatenate([Y_zigzag.flatten(), Cb_zigzag_4.flatten(), Cr_zigzag_4.flatten()])
compressed_4 = zlib.compress(combined_4.tobytes())
compressed_size_4 = len(compressed_4)

combined_2 = numpy.concatenate([Y_zigzag.flatten(), Cb_zigzag_2.flatten(), Cr_zigzag_2.flatten()])
compressed_2 = zlib.compress(combined_2.tobytes())
compressed_size_2 = len(compressed_2)

combined_0 = numpy.concatenate([Y_zigzag.flatten(), Cb_zigzag_0.flatten(), Cr_zigzag_0.flatten()])
compressed_0 = zlib.compress(combined_0.tobytes())
compressed_size_0 = len(compressed_0)

#
# 7'. Undo Zig Zag
#
# We can skip it in this exercise! We did Zig Zag only for analysis in step 8.
# You can continue with result from step 6. instead of implementing undo here.
#

#
# 6'. Nothing to do here   ¯\_(ツ)_/¯
#
# No conversion is really needed here, just proceed to the next step.
#

#
# 5'. Reverse division by quantisation matrix -- multiply
#

Y_dequant = Y_quant_rounded.astype(numpy.float32) * QY()

Cb_dequant_4 = Cb_quant_rounded_4.astype(numpy.float32) * QC()
Cr_dequant_4 = Cr_quant_rounded_4.astype(numpy.float32) * QC()

Cb_dequant_2 = Cb_quant_rounded_2.astype(numpy.float32) * QC()
Cr_dequant_2 = Cr_quant_rounded_2.astype(numpy.float32) * QC()

Cb_dequant_0 = Cb_quant_rounded_0.astype(numpy.float32) * QC()
Cr_dequant_0 = Cr_quant_rounded_0.astype(numpy.float32) * QC()

#
# 4'. Reverse DCT
#

Y_blocks = numpy.clip(numpy.array([idct2(block) for block in Y_dequant]), 0, 255).astype(numpy.uint8)

Cb_blocks_4 = numpy.clip(numpy.array([idct2(block) for block in Cb_dequant_4]), 0, 255).astype(numpy.uint8)
Cr_blocks_4 = numpy.clip(numpy.array([idct2(block) for block in Cr_dequant_4]), 0, 255).astype(numpy.uint8)

Cb_blocks_2 = numpy.clip(numpy.array([idct2(block) for block in Cb_dequant_2]), 0, 255).astype(numpy.uint8)
Cr_blocks_2 = numpy.clip(numpy.array([idct2(block) for block in Cr_dequant_2]), 0, 255).astype(numpy.uint8)

Cb_blocks_0 = numpy.clip(numpy.array([idct2(block) for block in Cb_dequant_0]), 0, 255).astype(numpy.uint8)
Cr_blocks_0 = numpy.clip(numpy.array([idct2(block) for block in Cr_dequant_0]), 0, 255).astype(numpy.uint8)

#
# 3'. Combine 8x8 blocks to original image
# 

def from_blocks(blocks, padded_shape, original_shape):
    h, w = padded_shape
    oh, ow = original_shape
    out = numpy.zeros((h, w), dtype=numpy.uint8)
    idx = 0
    for i in range(0, h, 8):
        for j in range(0, w, 8):
            out[i:i+8, j:j+8] = blocks[idx]
            idx += 1
    return out[:oh, :ow]


original_y_shape = Y.shape
original_cb_shape_4 = Cb_downsampled_4_4_4.shape
original_cr_shape_4 = Cr_downsampled_4_4_4.shape

original_cb_shape_2 = Cb_downsampled_4_2_2.shape
original_cr_shape_2 = Cr_downsampled_4_2_2.shape

original_cb_shape_0 = Cb_downsampled_4_2_0.shape
original_cr_shape_0 = Cr_downsampled_4_2_0.shape

Y_reconstructed = from_blocks(Y_blocks, Y_shape, original_y_shape)

Cb_decoded_4 = from_blocks(Cb_blocks_4, Cb_shape_4, original_cb_shape_4)
Cr_decoded_4 = from_blocks(Cr_blocks_4, Cr_shape_4, original_cr_shape_4)

Cb_decoded_2 = from_blocks(Cb_blocks_2, Cb_shape_2, original_cb_shape_2)
Cr_decoded_2 = from_blocks(Cr_blocks_2, Cr_shape_2, original_cr_shape_2)

Cb_decoded_0 = from_blocks(Cb_blocks_0, Cb_shape_0, original_cb_shape_0)
Cr_decoded_0 = from_blocks(Cr_blocks_0, Cr_shape_0, original_cr_shape_0)

#
# 2'. Upsampling on Cb and Cr channels
#
# TODO: implement (zad. 4)

Cb_upsampled_4 = Cb_decoded_4.copy()
Cr_upsampled_4 = Cr_decoded_4.copy()

Cb_upsampled_2 = cv2.resize(Cb_decoded_2, (int(width), int(height)), interpolation=cv2.INTER_LINEAR)
Cr_upsampled_2 = cv2.resize(Cr_decoded_2, (int(width), int(height)), interpolation=cv2.INTER_LINEAR)

Cb_upsampled_0 = cv2.resize(Cb_decoded_0, (int(width), int(height)), interpolation=cv2.INTER_LINEAR)
Cr_upsampled_0 = cv2.resize(Cr_decoded_0, (int(width), int(height)), interpolation=cv2.INTER_LINEAR)



#
# 1'. Convert YCbCr to RGB
#

image_ycrcb_restored_4 = cv2.merge((Y_reconstructed, Cb_upsampled_4, Cr_upsampled_4))
image_bgr_restored_4 = cv2.cvtColor(image_ycrcb_restored_4, cv2.COLOR_YCrCb2BGR)

image_ycrcb_restored_2 = cv2.merge((Y_reconstructed, Cb_upsampled_2, Cr_upsampled_2))
image_bgr_restored_2 = cv2.cvtColor(image_ycrcb_restored_2, cv2.COLOR_YCrCb2BGR)

image_ycrcb_restored_0 = cv2.merge((Y_reconstructed, Cb_upsampled_0, Cr_upsampled_0))
image_bgr_restored_0 = cv2.cvtColor(image_ycrcb_restored_0, cv2.COLOR_YCrCb2BGR)

#
# 0'. Save the decoded image -- as PPM or PNG
#
cv2.imwrite("lab4-po-jpeg-4:4:4.png", image_bgr_restored_4)
cv2.imwrite("lab4-po-jpeg-4:2:2.png", image_bgr_restored_2)
cv2.imwrite("lab4-po-jpeg-4:2:0.png", image_bgr_restored_0)

print("Original image (raw RGB) size:        " + str(image_rgb.nbytes) + " bytes")
print("Size of the product of point 8 (without DCT, exercise 4.): " + str(compressed_size_no_DCT) + " bytes")
print("Size of the product of point 8, with 4:4:4 (no sampling): " + str(compressed_size_4) + " bytes")
print("Size of the product of point 8, with 4:2:2 sampling: " + str(compressed_size_2) + " bytes")
print("Size of the product of point 8, with 4:2:0 sampling: " + str(compressed_size_0) + " bytes")

In [ ]:
#
# Display image
#

# Load both images
image_decompressed_4 = cv2.imread('lab4-po-jpeg-4:4:4.png')
image_decompressed_2 = cv2.imread('lab4-po-jpeg-4:2:2.png')
image_decompressed_0 = cv2.imread('lab4-po-jpeg-4:2:0.png')
image_original = cv2.imread('lab4-zad4-original.png')

# Convert from BGR to RGB for display
image_decompressed_rgb_4 = cv2.cvtColor(image_decompressed_4, cv2.COLOR_BGR2RGB)
image_decompressed_rgb_2 = cv2.cvtColor(image_decompressed_2, cv2.COLOR_BGR2RGB)
image_decompressed_rgb_0 = cv2.cvtColor(image_decompressed_0, cv2.COLOR_BGR2RGB)

# Create subplots
fig, axs = pyplot.subplots(2, 2, figsize=(20, 12))

axs[0, 0].imshow(image_original)
axs[0, 0].set_title("Original Image")

axs[0, 1].imshow(image_decompressed_rgb_4)
axs[0, 1].set_title("After JPEG with 4:4:4 (no sampling)")

axs[1, 0].imshow(image_decompressed_rgb_2)
axs[1, 0].set_title("After JPEG with 4:2:2 (every second px sampled)")

axs[1, 1].imshow(image_decompressed_rgb_0)
axs[1, 1].set_title("After JPEG with 4:2:0 (every fourth px sampled)")


# Show both images
pyplot.tight_layout()
pyplot.show()

# Obserwacje z zadań 4 i 5